In [9]:
#import streamlit as st
import pandas as pd

base_curves = pd.DataFrame(pd.read_csv('all_kfc_curves.csv'))

#curves_only = base_curves.drop('netspend',axis=1).reset_index()

def optimise_curve(df, budget, weeks):
    # create a new df to hold the results
    df_mid = pd.DataFrame()

    #calculate the number of 'units' that we have to assign
    units = budget/weeks/1000
    
    for col in df.columns:
        df_mid[col + '_incr'] = df[col].diff().fillna(0)
    
    df_mid = df_mid.filter(regex='_incr').melt(var_name = 'campaign').nlargest(round(units), 'value')

    budget_df = pd.DataFrame(df_mid['campaign'].value_counts()).reset_index()

    budget_df['campaign'] = budget_df['campaign'].str.replace('_incr', '')

    budget_df['budget'] = (budget_df['count']/(budget_df['count'].sum()))*budget

    budget_df = budget_df.drop('count',axis=1)

    budget_df['budget'] = budget_df['budget'].round()

    budget_df['wklybudget'] = (budget_df['budget']/weeks).round(-3)

    base_curves.set_index('netspend', inplace=True)

    budget_df['trans_driven'] = (budget_df.apply(lambda row: base_curves.loc[row['wklybudget'], row['campaign']], axis=1)).round()

    budget_df = (
        budget_df.assign(
            revenue_driven=(budget_df['trans_driven']*13.13).round(),
            ROI=lambda df: df['revenue_driven']/df['wklybudget']
        )).drop('wklybudget', axis=1)
    
    return budget_df

df_filtered = base_curves[['Crave_Cinema', 'Crave_Display']]

final_df = optimise_curve(df_filtered, weeks = 13, budget = 1100000).reset_index()

print(final_df)

   index       campaign    budget  trans_driven  revenue_driven       ROI
0      0  Crave_Display  763529.0       14922.0        195926.0  3.320780
1      1   Crave_Cinema  336471.0        7896.0        103674.0  3.987462


In [19]:
#settings
df = df_filtered
budget = 1100000
weeks = 11
col = 'Lunch_Social'

Lunch_TV_wklybdgt = 809000
Lunch_Social_wklybdgt = 191000


In [47]:
# Set netspend as index for easy lookup
#base_curves = pd.DataFrame(pd.read_csv('all_kfc_curves.csv'))
base_curves.set_index('netspend', inplace=True)

final_df['wklybudget'] = (final_df['budget']/weeks).round(-3)

# Create a new column in final_df
final_df['revenue_driven'] = final_df.apply(lambda row: base_curves.loc[row['wklybudget'], row['campaign']], axis=1)

print(final_df)

   index       campaign    budget  wklybudget  revenue_driven
0      0  Crave_Display  759000.0     69000.0        16305.34
1      1   Crave_Cinema  341000.0     31000.0         8587.52
